[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/hw-oh/wandb_e2e_demo/blob/main/models/image_detection/image_detection.ipynb)

In [1]:
!pip install -q wandb ultralytics wandb-workspaces

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.3/96.3 kB 6.2 MB/s eta 0:00:00


In [2]:

import wandb
import os
from google.colab import userdata

WANDB_API_KEY = userdata.get("WANDB_API_KEY")
WANDB_ENTITY = userdata.get("WANDB_ENTITY")
WANDB_PROJECT = userdata.get("WANDB_PROJECT")
WANDB_REGISTRY_NAME = userdata.get("WANDB_REGISTRY_NAME")

os.environ["WANDB_API_KEY"] = WANDB_API_KEY

wandb.login(key=WANDB_API_KEY)
print(f"Entity: {WANDB_ENTITY}")
print(f"Project: {WANDB_PROJECT}")

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: [wandb.login()] Using explicit session credentials for https://api.wandb.ai.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: hyunwoo-oh (wandb-korea) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Entity: wandb-korea
Project: e2e-cv-demo


# Unified Vision Demo — YOLOv8-seg로 Classification + Detection + Segmentation

## 개요

이 노트북은 **COCO128-seg** 데이터셋과 **YOLOv8n-seg** 모델을 사용하여
하나의 모델로 **Classification(분류)**, **Detection(탐지)**, **Segmentation(분할)** 을
모두 수행하면서 W&B의 핵심 기능을 전부 체험합니다.

### 왜 YOLOv8-seg인가?

YOLOv8-seg는 한 번의 추론으로 세 가지 결과를 동시에 제공합니다:
- **Classification**: 검출된 객체의 클래스 분류
- **Detection**: 바운딩 박스 좌표 + 클래스 + 신뢰도
- **Segmentation**: 인스턴스별 픽셀 마스크

### 모델 버전 관리 흐름

| 단계 | Artifact Alias | 설명 |
|------|---------------|------|
| Xavier 초기화 | `xavier-init` | 학습 전 랜덤 초기화 모델 (기준점) |
| Baseline 학습 | `baseline` | 기본 하이퍼파라미터로 학습한 모델 |
| Sweep 최적화 | `latest` | Sweep으로 찾은 최적 하이퍼파라미터로 학습한 모델 |

## 다루는 W&B 기능

| 기능 | 설명 |
|------|------|
| **Experiment Tracking** | 학습 메트릭 실시간 추적 (`wandb.init`, `wandb.log`, `wandb.config`) |
| **Media Logging** | BBox + Mask 인터랙티브 시각화 (`wandb.Image`) |
| **Tables** | 예측 결과 + mIoU 비교 (`wandb.Table`) |
| **Artifacts** | 데이터셋/모델 버저닝 및 계보(lineage) 추적 |
| **Model Registry** | 모델 등록 및 alias 관리 (UI에서 수행) |
| **Sweeps** | 베이지안 하이퍼파라미터 최적화 + 튜닝 효과 검증 |
| **Reports** | 프로그래밍 방식 실험 리포트 생성 |

In [3]:
from ultralytics import YOLO
from ultralytics.data.utils import check_det_dataset
from PIL import Image, ImageDraw
import numpy as np
import torch
import torch.nn as nn
import random
import glob
import csv

BASELINE_CONFIG = {
    "model_name": "yolov8n-seg",
    "dataset": "coco128-seg",
    "epochs": 30,
    "imgsz": 640,
    "lr0": 0.01,
    "batch": 16,
    "num_classes": 80,
}

COCO_CLASSES = [
    "person", "bicycle", "car", "motorcycle", "airplane", "bus", "train", "truck",
    "boat", "traffic light", "fire hydrant", "stop sign", "parking meter", "bench",
    "bird", "cat", "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra",
    "giraffe", "backpack", "umbrella", "handbag", "tie", "suitcase", "frisbee",
    "skis", "snowboard", "sports ball", "kite", "baseball bat", "baseball glove",
    "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
    "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange",
    "broccoli", "carrot", "hot dog", "pizza", "donut", "cake", "chair", "couch",
    "potted plant", "bed", "dining table", "toilet", "tv", "laptop", "mouse",
    "remote", "keyboard", "cell phone", "microwave", "oven", "toaster", "sink",
    "refrigerator", "book", "clock", "vase", "scissors", "teddy bear",
    "hair drier", "toothbrush",
]

CLASS_LABELS = {i: name for i, name in enumerate(COCO_CLASSES)}
MASK_CLASS_LABELS = {0: "background"}
MASK_CLASS_LABELS.update({i + 1: name for i, name in enumerate(COCO_CLASSES)})

ARTIFACT_NAME = "yolov8n-seg-coco128"

print(f"COCO classes: {len(COCO_CLASSES)}개")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
COCO classes: 80개


In [4]:
# === 헬퍼 함수 ===

def parse_yolo_seg_label(label_path):
    """YOLO-seg 라벨에서 class_id와 polygon을 파싱한다."""
    objects = []
    if not os.path.exists(label_path):
        return objects
    with open(label_path) as f:
        for line in f.readlines():
            parts = line.strip().split()
            if len(parts) < 7:
                continue
            cls_id = int(parts[0])
            coords = [float(x) for x in parts[1:]]
            xs, ys = coords[0::2], coords[1::2]
            objects.append({
                "cls_id": cls_id,
                "polygon": list(zip(xs, ys)),
                "bbox": {"x_min": min(xs), "y_min": min(ys), "x_max": max(xs), "y_max": max(ys)},
            })
    return objects


def polygons_to_mask(objects, img_w, img_h):
    """GT 오브젝트 리스트를 클래스별 마스크(2D int array)로 변환한다."""
    mask = np.zeros((img_h, img_w), dtype=np.int32)
    for obj in objects:
        pts = [(x * img_w, y * img_h) for x, y in obj["polygon"]]
        if len(pts) >= 3:
            pil_mask = Image.new("L", (img_w, img_h), 0)
            ImageDraw.Draw(pil_mask).polygon(pts, fill=1)
            mask[np.array(pil_mask) > 0] = obj["cls_id"] + 1
    return mask


def pred_to_mask(r, img_w, img_h):
    """YOLO 예측 결과를 클래스별 마스크(2D int array)로 변환한다."""
    mask = np.zeros((img_h, img_w), dtype=np.int32)
    if r.masks is None:
        return mask
    for poly_xy, box in zip(r.masks.xy, r.boxes):
        pts = [(float(p[0]), float(p[1])) for p in poly_xy]
        if len(pts) >= 3:
            pil_mask = Image.new("L", (img_w, img_h), 0)
            ImageDraw.Draw(pil_mask).polygon(pts, fill=1)
            mask[np.array(pil_mask) > 0] = int(box.cls) + 1
    return mask


def compute_miou(gt_mask, pred_mask):
    """GT 마스크와 예측 마스크의 mIoU를 계산한다."""
    classes = set(np.unique(gt_mask)) | set(np.unique(pred_mask))
    classes.discard(0)
    if not classes:
        return 0.0
    ious = []
    for c in classes:
        inter = np.logical_and(gt_mask == c, pred_mask == c).sum()
        union = np.logical_or(gt_mask == c, pred_mask == c).sum()
        if union > 0:
            ious.append(inter / union)
    return float(np.mean(ious)) if ious else 0.0


def build_prediction_table(yolo_model, image_paths):
    """모델 예측 결과를 wandb.Table로 생성한다."""
    table = wandb.Table(columns=[
        "Detection", "Segmentation", "Detected Classes",
        "Num Objects", "Avg Confidence", "mIoU",
    ])

    for img_path in image_paths:
        img = Image.open(img_path)
        img_w, img_h = img.size
        r = yolo_model(img_path, verbose=False)[0]

        # --- Detection column (인터랙티브 BBox) ---
        box_data, detected_classes, confs = [], set(), []
        for box in r.boxes:
            x1, y1, x2, y2 = box.xyxy[0].tolist()
            cls_id, conf = int(box.cls), float(box.conf)
            box_data.append({
                "position": {"minX": x1/img_w, "minY": y1/img_h, "maxX": x2/img_w, "maxY": y2/img_h},
                "class_id": cls_id,
                "scores": {"confidence": conf},
                "box_caption": f"{COCO_CLASSES[cls_id]} {conf:.2f}",
            })
            detected_classes.add(COCO_CLASSES[cls_id])
            confs.append(conf)
        det_img = wandb.Image(
            img,
            boxes={"predictions": {"box_data": box_data, "class_labels": CLASS_LABELS}} if box_data else {},
        )

        # --- Segmentation column (인터랙티브 마스크: 클래스별 on/off 가능) ---
        pred_mask = pred_to_mask(r, img_w, img_h)
        seg_img = wandb.Image(
            img,
            masks={"predictions": {"mask_data": pred_mask, "class_labels": MASK_CLASS_LABELS}},
        )

        # --- mIoU (GT 라벨 대비) ---
        label_path = img_path.replace("/images/", "/labels/").replace(".jpg", ".txt")
        gt_objects = parse_yolo_seg_label(label_path)
        gt_mask = polygons_to_mask(gt_objects, img_w, img_h)
        miou = compute_miou(gt_mask, pred_mask)

        avg_conf = float(np.mean(confs)) if confs else 0.0
        table.add_data(
            det_img, seg_img, ", ".join(sorted(detected_classes)),
            len(r.boxes), f"{avg_conf:.2%}", round(miou, 4),
        )

    return table


def log_training_metrics(results_save_dir):
    """YOLO 학습 결과 CSV를 W&B에 로깅한다."""
    csv_path = os.path.join(results_save_dir, "results.csv")
    if not os.path.exists(csv_path):
        return
    with open(csv_path) as f:
        reader = csv.DictReader(f)
        for row in reader:
            row = {k.strip(): v.strip() for k, v in row.items()}
            wandb.log({
                "epoch": int(row["epoch"]),
                "train/box_loss": float(row["train/box_loss"]),
                "train/seg_loss": float(row["train/seg_loss"]),
                "train/cls_loss": float(row["train/cls_loss"]),
                "train/dfl_loss": float(row["train/dfl_loss"]),
                "val/box_loss": float(row["val/box_loss"]),
                "val/seg_loss": float(row["val/seg_loss"]),
                "val/cls_loss": float(row["val/cls_loss"]),
                "val/dfl_loss": float(row["val/dfl_loss"]),
                "val/mAP50_box": float(row["metrics/mAP50(B)"]),
                "val/mAP50-95_box": float(row["metrics/mAP50-95(B)"]),
                "val/precision_box": float(row["metrics/precision(B)"]),
                "val/recall_box": float(row["metrics/recall(B)"]),
                "val/mAP50_mask": float(row["metrics/mAP50(M)"]),
                "val/mAP50-95_mask": float(row["metrics/mAP50-95(M)"]),
            })


print("헬퍼 함수 정의 완료")

헬퍼 함수 정의 완료


## 1. 데이터셋 준비 + Artifact 등록

In [5]:
data_info = check_det_dataset("coco128-seg.yaml")
DATASET_DIR = data_info["path"]
train_images = glob.glob(f"{DATASET_DIR}/images/train2017/*.jpg")
print(f"데이터셋 경로: {DATASET_DIR}")
print(f"Train 이미지: {len(train_images)}장")


WARNING ⚠️ Dataset 'coco128-seg.yaml' images not found, missing path '/content/datasets/coco128-seg/images/train2017'
Unzipping /content/datasets/coco128-seg.zip to /content/datasets/coco128-seg...: 100% ━━━━━━━━━━━━ 263/263 1.3Kfiles/s 0.2s
Dataset download success ✅ (0.6s), saved to /content/datasets

데이터셋 경로: /content/datasets/coco128-seg
Train 이미지: 128장


In [6]:
run = wandb.init(
    entity=WANDB_ENTITY,
    project=WANDB_PROJECT,
    config=BASELINE_CONFIG,
    job_type="data-versioning",
    name="coco128-seg-data-versioning",
)

artifact = wandb.Artifact(
    "coco128-seg",
    type="dataset",
    description="COCO128-seg dataset (128 images, 80 classes, instance segmentation labels)",
    metadata={
        "num_images": len(train_images),
        "num_classes": 80,
        "classes": COCO_CLASSES,
        "format": "YOLO-seg",
    },
)
artifact.add_dir(DATASET_DIR)
run.log_artifact(artifact)

# GT BBox + Mask 시각화 테이블
table = wandb.Table(columns=["GT Overlay", "Num Objects", "Classes"])
sample_images = random.sample(train_images, min(20, len(train_images)))

for img_path in sample_images:
    img = Image.open(img_path)
    img_w, img_h = img.size
    label_path = img_path.replace("/images/", "/labels/").replace(".jpg", ".txt")
    gt_objects = parse_yolo_seg_label(label_path)

    box_data = []
    for obj in gt_objects:
        bb = obj["bbox"]
        box_data.append({
            "position": {"minX": bb["x_min"], "minY": bb["y_min"], "maxX": bb["x_max"], "maxY": bb["y_max"]},
            "class_id": obj["cls_id"],
            "box_caption": COCO_CLASSES[obj["cls_id"]] if obj["cls_id"] < len(COCO_CLASSES) else str(obj["cls_id"]),
        })

    gt_mask = polygons_to_mask(gt_objects, img_w, img_h)
    gt_img = wandb.Image(
        img,
        boxes={"ground_truth": {"box_data": box_data, "class_labels": CLASS_LABELS}} if box_data else {},
        masks={"ground_truth": {"mask_data": gt_mask, "class_labels": MASK_CLASS_LABELS}},
    )
    class_names = list({COCO_CLASSES[o["cls_id"]] for o in gt_objects})
    table.add_data(gt_img, len(gt_objects), ", ".join(class_names))

wandb.log({"dataset_preview": table})
wandb.finish()
print("데이터셋 Artifact + GT 시각화 로깅 완료!")

wandb: Adding directory to artifact (/content/datasets/coco128-seg)... Done. 0.2s


데이터셋 Artifact + GT 시각화 로깅 완료!


## 2. Xavier 초기화 모델 등록

학습을 시작하기 전, **Xavier 초기화**만 적용한 모델을 Artifact로 등록합니다.
이 모델은 아무것도 학습하지 않은 상태이므로, 학습 전후의 성능 차이를 명확하게 보여주는 기준점 역할을 합니다.

In [8]:
run = wandb.init(
    entity=WANDB_ENTITY,
    project=WANDB_PROJECT,
    config=BASELINE_CONFIG,
    job_type="model-init",
    name="yolov8n-seg-xavier-init",
)

data_artifact = run.use_artifact(f"{WANDB_ENTITY}/{WANDB_PROJECT}/coco128-seg:latest")

init_model = YOLO("yolov8n-seg.yaml")

for m in init_model.model.modules():
    if isinstance(m, nn.Conv2d):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            nn.init.zeros_(m.bias)
    elif isinstance(m, nn.BatchNorm2d):
        nn.init.ones_(m.weight)
        nn.init.zeros_(m.bias)

XAVIER_PT = "yolov8n-seg-xavier.pt"
torch.save({"model": init_model.model}, XAVIER_PT)

xavier_artifact = wandb.Artifact(
    ARTIFACT_NAME,
    type="model",
    description="YOLOv8n-seg with Xavier initialization (no training)",
    metadata={
        "model_type": "yolo-seg",
        "model_architecture": "yolov8n-seg",
        "dataset": "coco128-seg",
        "num_classes": 80,
        "classes": COCO_CLASSES,
        "framework": "ultralytics",
        "input_size": [3, 640, 640],
        "initialization": "xavier_uniform",
        "trained": False,
        "best_mAP50": 0.0,
        "best_mAP50_mask": 0.0,
    },
)
xavier_artifact.add_file(XAVIER_PT, name="best.pt")
run.log_artifact(xavier_artifact, aliases=["xavier-init"])

# Model Registry 등록 — UI에서 직접 수행합니다
# run.link_artifact(
#     xavier_artifact,
#     f"{WANDB_REGISTRY_NAME}/coco128-vision",
#     aliases=["xavier-init"],
# )

print("Xavier 초기화 모델 Artifact 등록 완료! (alias: xavier-init)")
wandb.finish()

Xavier 초기화 모델 Artifact 등록 완료! (alias: xavier-init)


In [10]:
run = wandb.init(
    entity=WANDB_ENTITY,
    project=WANDB_PROJECT,
    config={**BASELINE_CONFIG, "initialization": "xavier_uniform", "pretrained": False},
    job_type="training",
    name="xavier-trained-yolov8n-seg",
    tags=["xavier", "from-scratch"],
)

xavier_model = YOLO("yolov8n-seg-xavier.pt")

xavier_results = xavier_model.train(
    data="coco128-seg.yaml",
    epochs=BASELINE_CONFIG["epochs"],
    imgsz=BASELINE_CONFIG["imgsz"],
    lr0=BASELINE_CONFIG["lr0"],
    batch=BASELINE_CONFIG["batch"],
    project="runs/segment",
    name="xavier_trained",
    exist_ok=True,
    verbose=True,
)

log_training_metrics(xavier_results.save_dir)

XAVIER_BEST_PT = os.path.join(xavier_results.save_dir, "weights/best.pt")
xavier_metrics = xavier_results.results_dict
xavier_mAP50_box = xavier_metrics.get("metrics/mAP50(B)", 0)
xavier_mAP50_mask = xavier_metrics.get("metrics/mAP50(M)", 0)

xavier_best_model = YOLO(XAVIER_BEST_PT)
eval_images_xavier = random.sample(train_images, min(30, len(train_images)))
pred_table_xavier = build_prediction_table(xavier_best_model, eval_images_xavier)
wandb.log({"prediction_table": pred_table_xavier})

xavier_trained_artifact = wandb.Artifact(
    ARTIFACT_NAME,
    type="model",
    description="YOLOv8n-seg trained from Xavier init (from scratch) on COCO128-seg",
    metadata={
        "model_type": "yolo-seg",
        "model_architecture": "yolov8n-seg",
        "dataset": "coco128-seg",
        "num_classes": 80,
        "classes": COCO_CLASSES,
        "framework": "ultralytics",
        "input_size": [3, 640, 640],
        "epochs": BASELINE_CONFIG["epochs"],
        "initialization": "xavier_uniform",
        "pretrained": False,
        "best_mAP50": xavier_mAP50_box,
        "best_mAP50_mask": xavier_mAP50_mask,
        "sweep_tuned": False,
    },
)
xavier_trained_artifact.add_file(XAVIER_BEST_PT, name="best.pt")
run.log_artifact(xavier_trained_artifact, aliases=["xavier-trained"])

print(f"\nXavier 초기화 모델 학습 완료!")
print(f"  mAP50 (Box):  {xavier_mAP50_box:.4f}")
print(f"  mAP50 (Mask): {xavier_mAP50_mask:.4f}")
print("Xavier 학습 모델 Artifact 등록 완료! (alias: xavier-trained)")

wandb.finish()

Ultralytics 8.4.14 🚀 Python-3.12.12 torch-2.10.0+cu128 CUDA:0 (Tesla T4, 14913MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=coco128-seg.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=30, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n-seg-xavier.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=xavier_trained, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, persp

Exception in thread Thread-54 (plot_images):
Traceback (most recent call last):
  File "/usr/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.12/threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.12/dist-packages/ultralytics/utils/plotting.py", line 796, in plot_images
    annotator.box_label(box, label, color=color)
  File "/usr/local/lib/python3.12/dist-packages/ultralytics/utils/plotting.py", line 326, in box_label
    ) if multi_points else self.draw.rectangle(box, width=self.lw, outline=color)
                           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/PIL/ImageDraw.py", line 398, in rectangle
    self.draw.draw_rectangle(xy, ink, 0, width)
ValueError: x1 must be greater than or equal to x0


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 50% ━━━━━━────── 2/4 5.6s/it 5.7s<11.2s

Exception in thread Thread-56 (plot_images):
Traceback (most recent call last):
  File "/usr/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.12/threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.12/dist-packages/ultralytics/utils/plotting.py", line 796, in plot_images
    annotator.box_label(box, label, color=color)
  File "/usr/local/lib/python3.12/dist-packages/ultralytics/utils/plotting.py", line 326, in box_label
    ) if multi_points else self.draw.rectangle(box, width=self.lw, outline=color)
                           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/PIL/ImageDraw.py", line 398, in rectangle
    self.draw.draw_rectangle(xy, ink, 0, width)
ValueError: x1 must be greater than or equal to x0


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 4/4 2.9s/it 11.7s


Exception in thread Thread-58 (plot_images):
Traceback (most recent call last):
  File "/usr/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.12/threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.12/dist-packages/ultralytics/utils/plotting.py", line 796, in plot_images
    annotator.box_label(box, label, color=color)
  File "/usr/local/lib/python3.12/dist-packages/ultralytics/utils/plotting.py", line 326, in box_label
    ) if multi_points else self.draw.rectangle(box, width=self.lw, outline=color)
                           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/PIL/ImageDraw.py", line 398, in rectangle
    self.draw.draw_rectangle(xy, ink, 0, width)
ValueError: x1 must be greater than or equal to x0


                   all        128        929          0          0          0          0          0          0          0          0
                person         61        254          0          0          0          0          0          0          0          0
               bicycle          3          6          0          0          0          0          0          0          0          0
                   car         12         46          0          0          0          0          0          0          0          0
            motorcycle          4          5          0          0          0          0          0          0          0          0
              airplane          5          6          0          0          0          0          0          0          0          0
                   bus          5          7          0          0          0          0          0          0          0          0
                 train          3          3          0          0   

epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
train/box_loss,▂▅▄▄▃▆▄▄▄▆▅▇█▅▆▆▄▇▅▇▃▂▁▁▃▄▂▄▃▂
train/cls_loss,▂▃▂▃▃▃▂▂▂▂▂▂▁▂▁▂▁▁▂▁▇███▇██▇▇█
train/dfl_loss,▆█▇█▇▇▆▆▆▅▅▅▂▄▄▃▄▂▁▁▃▃▆▇▂▅▄▃▃▃
train/seg_loss,▇█▆█▅▆▇▅▇▅▅▄▂▄▄▃▄▂▁▂▅▃▃▄▅▄▃▁▃▄
val/box_loss,██████████████▇▇▆▆▆▅▅▄▄▃▃▂▂▂▁▁
val/dfl_loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▃▃▄▅▆▇██
val/mAP50-95_box,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val/mAP50-95_mask,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val/mAP50_box,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
+5,...


## 3. Baseline 학습 — 기본 하이퍼파라미터

기본 하이퍼파라미터(`lr0=0.01`, `batch=16`)로 YOLOv8n-seg를 학습합니다.
이 결과를 기준점(baseline)으로 삼아, Sweep을 통해 얼마나 개선되는지 비교합니다.

In [ ]:
os.environ["WANDB_DISABLED"] = "true"

run = wandb.init(
    entity=WANDB_ENTITY,
    project=WANDB_PROJECT,
    config=BASELINE_CONFIG,
    job_type="training",
    name="baseline-yolov8n-seg",
    tags=["baseline"],
)

data_artifact = run.use_artifact(f"{WANDB_ENTITY}/{WANDB_PROJECT}/coco128-seg:latest")

model = YOLO("yolov8n-seg.pt")

results = model.train(
    data="coco128-seg.yaml",
    epochs=BASELINE_CONFIG["epochs"],
    imgsz=BASELINE_CONFIG["imgsz"],
    lr0=BASELINE_CONFIG["lr0"],
    batch=BASELINE_CONFIG["batch"],
    project="runs/segment",
    name="baseline",
    exist_ok=True,
    verbose=True,
)

log_training_metrics(results.save_dir)

BASELINE_BEST_PT = os.path.join(results.save_dir, "weights/best.pt")
baseline_metrics = results.results_dict
baseline_mAP50_box = baseline_metrics.get("metrics/mAP50(B)", 0)
baseline_mAP50_mask = baseline_metrics.get("metrics/mAP50(M)", 0)

print(f"Baseline 학습 완료!")
print(f"  mAP50 (Box):  {baseline_mAP50_box:.4f}")
print(f"  mAP50 (Mask): {baseline_mAP50_mask:.4f}")

In [ ]:
best_model = YOLO(BASELINE_BEST_PT)
eval_images = random.sample(train_images, min(30, len(train_images)))

pred_table = build_prediction_table(best_model, eval_images)
wandb.log({"prediction_table": pred_table})
print(f"Prediction Table ({len(eval_images)}장) 로깅 완료!")

In [ ]:
baseline_artifact = wandb.Artifact(
    ARTIFACT_NAME,
    type="model",
    description="YOLOv8n-seg baseline on COCO128-seg (default hyperparameters)",
    metadata={
        "model_type": "yolo-seg",
        "model_architecture": "yolov8n-seg",
        "dataset": "coco128-seg",
        "num_classes": 80,
        "classes": COCO_CLASSES,
        "framework": "ultralytics",
        "input_size": [3, 640, 640],
        "epochs": BASELINE_CONFIG["epochs"],
        "best_mAP50": baseline_mAP50_box,
        "best_mAP50_mask": baseline_mAP50_mask,
        "sweep_tuned": False,
    },
)
baseline_artifact.add_file(BASELINE_BEST_PT, name="best.pt")
run.log_artifact(baseline_artifact, aliases=["baseline"])

# Model Registry 등록 — UI에서 직접 수행합니다
# run.link_artifact(
#     baseline_artifact,
#     f"{WANDB_REGISTRY_NAME}/coco128-vision",
#     aliases=["baseline"],
# )

print("Baseline 모델 Artifact 등록 완료! (alias: baseline)")
wandb.finish()

## 4. W&B Sweep — 하이퍼파라미터 최적화

W&B Sweeps의 **베이지안 최적화**를 사용하여 최적의 하이퍼파라미터를 탐색합니다.
여러 번의 Sweep run을 통해 최적 조합을 찾아가는 과정을 관찰하세요.

**탐색 파라미터:**
- `lr0`: 초기 학습률 (0.001 ~ 0.02)
- `lrf`: 최종 학습률 비율 (0.01 ~ 0.2)
- `momentum`: SGD 모멘텀 (0.85 ~ 0.98)
- `weight_decay`: 가중치 감쇠 (0.0001 ~ 0.001)
- `warmup_epochs`: 웜업 에포크 수 (1, 2, 3)
- `mosaic`: 모자이크 증강 비율 (0.5 ~ 1.0)

**최적화 목표:** `val/mAP50_box` 최대화

In [ ]:
sweep_config = {
    "method": "bayes",
    "metric": {"name": "val/mAP50_box", "goal": "maximize"},
    "parameters": {
        "lr0": {"min": 0.001, "max": 0.02},
        "lrf": {"min": 0.01, "max": 0.2},
        "momentum": {"min": 0.85, "max": 0.98},
        "weight_decay": {"min": 0.0001, "max": 0.001},
        "warmup_epochs": {"values": [1, 2, 3]},
        "mosaic": {"min": 0.5, "max": 1.0},
    },
}

SWEEP_EPOCHS = 10


def sweep_train():
    run = wandb.init()
    cfg = wandb.config

    sweep_model = YOLO("yolov8n-seg.pt")
    sweep_results = sweep_model.train(
        data="coco128-seg.yaml",
        epochs=SWEEP_EPOCHS,
        imgsz=640,
        lr0=cfg.lr0,
        lrf=cfg.lrf,
        momentum=cfg.momentum,
        weight_decay=cfg.weight_decay,
        warmup_epochs=cfg.warmup_epochs,
        mosaic=cfg.mosaic,
        batch=16,
        project="runs/segment_sweep",
        name=f"sweep_{run.id}",
        exist_ok=True,
        verbose=False,
    )

    metrics = sweep_results.results_dict
    wandb.log({
        "val/mAP50_box": metrics.get("metrics/mAP50(B)", 0),
        "val/mAP50-95_box": metrics.get("metrics/mAP50-95(B)", 0),
        "val/precision_box": metrics.get("metrics/precision(B)", 0),
        "val/recall_box": metrics.get("metrics/recall(B)", 0),
        "val/mAP50_mask": metrics.get("metrics/mAP50(M)", 0),
        "val/mAP50-95_mask": metrics.get("metrics/mAP50-95(M)", 0),
    })
    wandb.finish()


sweep_id = wandb.sweep(sweep_config, project=WANDB_PROJECT, entity=WANDB_ENTITY)
wandb.agent(sweep_id, function=sweep_train, count=10)
print("Sweep 완료!")

## 5. Sweep 최적 하이퍼파라미터로 Full Training

Sweep에서 찾은 최적의 하이퍼파라미터로 full epoch 학습을 실행합니다.
Baseline과 비교하여 Sweep의 효과를 확인합니다.

In [ ]:
api = wandb.Api()
sweep = api.sweep(f"{WANDB_ENTITY}/{WANDB_PROJECT}/{sweep_id}")
best_run = sweep.best_run()
best_config = best_run.config

print("Sweep 최적 하이퍼파라미터:")
for k, v in best_config.items():
    print(f"  {k}: {v}")
print(f"  Best mAP50 (Box): {best_run.summary.get('val/mAP50_box', 'N/A')}")

In [ ]:
tuned_config = {**BASELINE_CONFIG, **best_config}
tuned_config["sweep_tuned"] = True
tuned_config["sweep_id"] = sweep_id

run = wandb.init(
    entity=WANDB_ENTITY,
    project=WANDB_PROJECT,
    config=tuned_config,
    job_type="training",
    name="tuned-yolov8n-seg",
    tags=["sweep-tuned"],
)

tuned_model = YOLO("yolov8n-seg.pt")
tuned_results = tuned_model.train(
    data="coco128-seg.yaml",
    epochs=BASELINE_CONFIG["epochs"],
    imgsz=640,
    lr0=best_config.get("lr0", 0.01),
    lrf=best_config.get("lrf", 0.01),
    momentum=best_config.get("momentum", 0.937),
    weight_decay=best_config.get("weight_decay", 0.0005),
    warmup_epochs=best_config.get("warmup_epochs", 3),
    mosaic=best_config.get("mosaic", 1.0),
    batch=16,
    project="runs/segment",
    name="tuned",
    exist_ok=True,
    verbose=True,
)

log_training_metrics(tuned_results.save_dir)

TUNED_BEST_PT = os.path.join(tuned_results.save_dir, "weights/best.pt")
tuned_metrics = tuned_results.results_dict
tuned_mAP50_box = tuned_metrics.get("metrics/mAP50(B)", 0)
tuned_mAP50_mask = tuned_metrics.get("metrics/mAP50(M)", 0)

print(f"Tuned 학습 완료!")
print(f"  mAP50 (Box):  {tuned_mAP50_box:.4f}")
print(f"  mAP50 (Mask): {tuned_mAP50_mask:.4f}")

In [ ]:
tuned_best_model = YOLO(TUNED_BEST_PT)
eval_images_tuned = random.sample(train_images, min(30, len(train_images)))

pred_table_tuned = build_prediction_table(tuned_best_model, eval_images_tuned)
wandb.log({"prediction_table": pred_table_tuned})
print(f"Tuned Prediction Table ({len(eval_images_tuned)}장) 로깅 완료!")

In [ ]:
print("=" * 50)
print("Baseline vs Sweep-Tuned 비교")
print("=" * 50)

box_diff = tuned_mAP50_box - baseline_mAP50_box
mask_diff = tuned_mAP50_mask - baseline_mAP50_mask

print(f"{'Metric':<20} {'Baseline':>10} {'Tuned':>10} {'Diff':>10}")
print("-" * 50)
print(f"{'mAP50 (Box)':<20} {baseline_mAP50_box:>10.4f} {tuned_mAP50_box:>10.4f} {box_diff:>+10.4f}")
print(f"{'mAP50 (Mask)':<20} {baseline_mAP50_mask:>10.4f} {tuned_mAP50_mask:>10.4f} {mask_diff:>+10.4f}")

wandb.log({
    "comparison/baseline_mAP50_box": baseline_mAP50_box,
    "comparison/tuned_mAP50_box": tuned_mAP50_box,
    "comparison/improvement_mAP50_box": box_diff,
    "comparison/baseline_mAP50_mask": baseline_mAP50_mask,
    "comparison/tuned_mAP50_mask": tuned_mAP50_mask,
    "comparison/improvement_mAP50_mask": mask_diff,
})

In [ ]:
final_mAP50 = max(tuned_mAP50_box, baseline_mAP50_box)
final_mAP50_mask = max(tuned_mAP50_mask, baseline_mAP50_mask)
final_pt = TUNED_BEST_PT if tuned_mAP50_box >= baseline_mAP50_box else BASELINE_BEST_PT
is_tuned_better = tuned_mAP50_box >= baseline_mAP50_box

print(f"최종 모델: {'Tuned' if is_tuned_better else 'Baseline'} (mAP50 Box: {final_mAP50:.4f})")

model_artifact = wandb.Artifact(
    ARTIFACT_NAME,
    type="model",
    description=f"YOLOv8n-seg {'sweep-tuned' if is_tuned_better else 'baseline'} on COCO128-seg",
    metadata={
        "model_type": "yolo-seg",
        "model_architecture": "yolov8n-seg",
        "dataset": "coco128-seg",
        "num_classes": 80,
        "classes": COCO_CLASSES,
        "framework": "ultralytics",
        "input_size": [3, 640, 640],
        "epochs": BASELINE_CONFIG["epochs"],
        "best_mAP50": final_mAP50,
        "best_mAP50_mask": final_mAP50_mask,
        "sweep_tuned": is_tuned_better,
    },
)
model_artifact.add_file(final_pt, name="best.pt")
run.log_artifact(model_artifact, aliases=["latest"])

# Model Registry 등록 — UI에서 직접 수행합니다
# run.link_artifact(
#     model_artifact,
#     f"{WANDB_REGISTRY_NAME}/coco128-vision",
#     aliases=["staging"],
# )

print("최종 모델 Artifact 등록 완료! (alias: latest)")
wandb.finish()

## 6. W&B Report 생성

In [ ]:
import wandb_workspaces.reports.v2 as wr

report = wr.Report(
    entity=WANDB_ENTITY,
    project=WANDB_PROJECT,
    title="Unified Vision — YOLOv8n-seg 실험 결과 리포트",
    description="YOLOv8n-seg COCO128 학습 + Sweep 하이퍼파라미터 튜닝 결과",
)

report.blocks = [
    wr.TableOfContents(),

    wr.H1("1. 실험 개요"),
    wr.P(
        "COCO128-seg 데이터셋에 대한 YOLOv8n-seg 통합 비전 모델 실험 결과를 정리합니다. "
        "하나의 모델로 Classification, Detection, Segmentation을 모두 수행합니다. "
        "W&B의 Experiment Tracking, Artifacts, Sweeps, Model Registry, Media Logging 기능을 활용하였습니다."
    ),

    wr.H1("2. Baseline 학습 결과"),
    wr.PanelGrid(
        runsets=[wr.Runset(entity=WANDB_ENTITY, project=WANDB_PROJECT)],
        panels=[
            wr.LinePlot(title="Box Loss (Train)", x="epoch", y=["train/box_loss"]),
            wr.LinePlot(title="Seg Loss (Train)", x="epoch", y=["train/seg_loss"]),
            wr.LinePlot(title="mAP@0.5 (Box)", x="epoch", y=["val/mAP50_box"]),
            wr.LinePlot(title="mAP@0.5 (Mask)", x="epoch", y=["val/mAP50_mask"]),
            wr.LinePlot(title="Precision (Box)", x="epoch", y=["val/precision_box"]),
            wr.LinePlot(title="Recall (Box)", x="epoch", y=["val/recall_box"]),
        ],
    ),

    wr.H1("3. Sweep 분석"),
    wr.P("Bayesian 최적화를 통한 하이퍼파라미터 탐색 결과:"),
    wr.PanelGrid(
        runsets=[wr.Runset(entity=WANDB_ENTITY, project=WANDB_PROJECT)],
        panels=[
            wr.ParallelCoordinatesPlot(
                columns=[
                    wr.ParallelCoordinatesPlotColumn(metric="c::lr0"),
                    wr.ParallelCoordinatesPlotColumn(metric="c::lrf"),
                    wr.ParallelCoordinatesPlotColumn(metric="c::momentum"),
                    wr.ParallelCoordinatesPlotColumn(metric="c::weight_decay"),
                    wr.ParallelCoordinatesPlotColumn(metric="c::mosaic"),
                    wr.ParallelCoordinatesPlotColumn(metric="val/mAP50_box"),
                ],
            ),
            wr.ScalarChart(title="Best mAP@0.5 (Box)", metric="val/mAP50_box"),
            wr.ScalarChart(title="Best mAP@0.5 (Mask)", metric="val/mAP50_mask"),
            wr.BarPlot(title="mAP@0.5 by Run", metrics=["val/mAP50_box"]),
        ],
    ),

    wr.H1("4. Baseline vs Sweep-Tuned 비교"),
    wr.P(
        f"Baseline mAP50 (Box): {baseline_mAP50_box:.4f} → "
        f"Tuned mAP50 (Box): {tuned_mAP50_box:.4f} "
        f"(차이: {box_diff:+.4f})"
    ),

    wr.H1("5. 다음 단계"),
    wr.P(
        "최적 모델을 Model Registry의 'production' alias로 승격하여 "
        "Automation → GitHub Actions → Streamlit 배포 파이프라인을 트리거합니다."
    ),
]

report.save()
print(f"Report 생성 완료! URL: {report.url}")

## 7. Production 승격 (Automation 트리거)

아래 셀을 실행하면 Model Registry에서 최신 모델을 `production` alias로 승격합니다.

W&B Automation이 설정되어 있으면:
1. `production` alias 추가 이벤트 발생
2. Webhook → GitHub `repository_dispatch` 트리거
3. GitHub Actions가 `deployment.json` 업데이트 → git push
4. Streamlit Cloud 앱이 새 모델로 자동 배포

**참고**: W&B UI에서 수동으로 `production` alias를 추가해도 동일하게 동작합니다.

In [ ]:
# api = wandb.Api()
# artifact_path = f"{WANDB_ENTITY}/{WANDB_PROJECT}/{ARTIFACT_NAME}:latest"
#
# try:
#     art = api.artifact(artifact_path)
#     art.aliases.append("production")
#     art.save()
#     print(f"'{artifact_path}'에 'production' alias 추가 완료!")
#     print("W&B Automation이 설정되어 있으면 배포 파이프라인이 자동으로 트리거됩니다.")
# except Exception as e:
#     print(f"Production 승격 실패: {e}")
#     print("W&B UI에서 수동으로 'production' alias를 추가해 주세요.")

In [ ]:
wandb.finish()
print("모든 W&B 리소스가 정리되었습니다. 데모 완료!")